# Classic Monty Hall Bayesian Network

authors:<br>
Jacob Schreiber [<a href="mailto:jmschreiber91@gmail.com">jmschreiber91@gmail.com</a>]<br>
Nicholas Farn [<a href="mailto:nicholasfarn@gmail.com">nicholasfarn@gmail.com</a>]

Lets test out the Bayesian Network framework to produce the Monty Hall problem, but modified a little. The Monty Hall problem is basically a game show where a guest chooses one of three doors to open, with an unknown one having a prize behind it. Monty then opens another non-chosen door without a prize behind it, and asks the guest if they would like to change their answer. Many people were surprised to find that if the guest changed their answer, there was a 66% chance of success as opposed to a 50% as might be expected if there were two doors.

This can be modelled as a Bayesian network with three nodes-- guest, prize, and Monty, each over the domain of door 'A', 'B', 'C'. Monty is dependent on both guest and prize, in that it can't be either of them. Lets extend this a little bit to say the guest has an untrustworthy friend whose answer he will not go with.

In [5]:
# !c:\Users\fatib\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip
%pip install pomegranate==0.14.5

ERROR: Could not find a version that satisfies the requirement pomegranate==0.14.5 (from versions: 0.0.1, 0.0.2, 0.1.0, 0.1.1, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.2.9, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.5, 0.3.6, 0.3.7, 0.4.0, 0.5.0, 0.5.1, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.6, 0.7.7, 0.8.0, 0.8.1, 0.9.0, 0.10.0, 0.11.0, 0.11.1, 0.11.2, 0.12.0, 0.12.2, 0.13.0, 0.13.1rc0, 0.13.2, 0.13.3, 0.13.4, 0.13.5, 0.14.0, 0.14.2, 0.14.3, 0.14.4, 0.14.7, 0.14.8)
ERROR: No matching distribution found for pomegranate==0.14.5
Note: you may need to restart the kernel to use updated packages.


In [6]:
import math
from pomegranate import *

ModuleNotFoundError: No module named 'pomegranate'

Let's create the distributions for the guest and the prize. Note that both distributions are independent of one another.

In [ ]:
guest = DiscreteDistribution( { 'A': 1./3, 'B': 1./3, 'C': 1./3 } )
prize = DiscreteDistribution( { 'A': 1./3, 'B': 1./3, 'C': 1./3 } )

Now let's create the conditional probability table for our Monty. The table is dependent on both the guest and the prize.

In [ ]:
monty = ConditionalProbabilityTable(
	[[ 'A', 'A', 'A', 0.0 ],
	 [ 'A', 'A', 'B', 0.5 ],
	 [ 'A', 'A', 'C', 0.5 ],
	 [ 'A', 'B', 'A', 0.0 ],
	 [ 'A', 'B', 'B', 0.0 ],
	 [ 'A', 'B', 'C', 1.0 ],
	 [ 'A', 'C', 'A', 0.0 ],
	 [ 'A', 'C', 'B', 1.0 ],
	 [ 'A', 'C', 'C', 0.0 ],
	 [ 'B', 'A', 'A', 0.0 ],
	 [ 'B', 'A', 'B', 0.0 ],
	 [ 'B', 'A', 'C', 1.0 ],
	 [ 'B', 'B', 'A', 0.5 ],
	 [ 'B', 'B', 'B', 0.0 ],
	 [ 'B', 'B', 'C', 0.5 ],
	 [ 'B', 'C', 'A', 1.0 ],
	 [ 'B', 'C', 'B', 0.0 ],
	 [ 'B', 'C', 'C', 0.0 ],
	 [ 'C', 'A', 'A', 0.0 ],
	 [ 'C', 'A', 'B', 1.0 ],
	 [ 'C', 'A', 'C', 0.0 ],
	 [ 'C', 'B', 'A', 1.0 ],
	 [ 'C', 'B', 'B', 0.0 ],
	 [ 'C', 'B', 'C', 0.0 ],
	 [ 'C', 'C', 'A', 0.5 ],
	 [ 'C', 'C', 'B', 0.5 ],
	 [ 'C', 'C', 'C', 0.0 ]], [guest, prize] )
guest1 = ConditionalProbabilityTable(
	[[ 'A', 'A', 0.0 ],
	 [ 'A', 'B', 0.5 ],
	 [ 'A', 'C', 0.5 ],
	 [ 'B', 'A', 0.5 ],
	 [ 'B', 'B', 0.0 ],
	 [ 'B', 'C', 0.5 ],
	 [ 'C', 'A', 0.5 ],
	 [ 'C', 'B', 0.5 ],
	 [ 'C', 'C', 0.0 ]], [monty] )

Now lets create the states for the bayesian network.

In [ ]:
s1 = State( guest, name="guest" )
s2 = State( prize, name="prize" )
s3 = State( monty, name="monty" )
s4 = State( guest1, name="guest1" )

Then the bayesian network itself, adding the states in after.

In [ ]:
network = BayesianNetwork( "test" )
network.add_states( s1, s2, s3, s4 )

Then the transitions.

In [ ]:
network.add_transition( s1, s3 )
network.add_transition( s2, s3 )
network.add_transition( s3, s4 )

With a "bake" to finalize the structure of our network.

In [ ]:
network.bake()

Now we can check the possible states in our network.

In [ ]:
print("\t".join([ state.name for state in network.states ]))

Now we can see what happens to our network when our Guest chooses 'A'.

In [ ]:
observations = { 'guest' : 'A' }
beliefs = map( str, network.predict_proba( observations ) )
print("\n".join(f"{state.name}\t{belief}" for state, belief in zip(network.states, beliefs)))

Now our host chooses 'B'. (note that prize goes to 66% if you switch)

In [ ]:
observations = { 'guest' : 'A', 'monty' : 'B' }
beliefs = map( str, network.predict_proba( observations ) )
print("\n".join(f"{state.name}\t{belief}" for state, belief in zip(network.states, beliefs)))

We can also see what happens if our host simply chooses 'B'.

In [ ]:
observations = { 'monty' : 'B' }
beliefs = map( str, network.predict_proba( observations ) )
print("\n".join(f"{state.name}\t{belief}" for state, belief in zip(network.states, beliefs)))